# prologue

### set up notebook and load package

In [257]:
import io
import zipfile
import urllib
import numpy as np
import pandas as pd
from datetime import datetime
import math
import julian
import re

In [260]:
file = '2014.csv'
archive = zipfile.ZipFile('CHIRPS/source_datafiles/mhtech.zip', 'r')
lines = archive.read(file).decode("utf-8").split('\n')
archive.close()

# convert strings to var lists - deal with some free text issues
lines = [lines[i].replace("Bahamas, The", "Bahamas").replace("male, unsure", "male unsure").split(',') for i in range(len(lines))]
names = [nm.replace('"', '') for nm in lines[0]]
lines = lines[1:]
lines.pop() # empty final row
mhtech = pd.DataFrame(lines, columns=names)

# not useful, turns out state only valid for USA
mhtech.drop(['Timestamp', 'state'], 1, inplace=True)

# rename columns for ease
ren = {old : new for old, new in zip(mhtech.columns, [lc.lower() for lc in mhtech.columns])}
mhtech.rename(columns=ren, inplace=True)

# standardise some other responses, getting rid of 'apos in "Don't know"
for c in mhtech.columns:
    mhtech[c] = mhtech[c].str.replace("Don't know", 'Not sure')
    mhtech[c] = mhtech[c].str.replace('"', '')
    mhtech[c] = mhtech[c].str.replace('Maybe', 'Not sure')
    mhtech[c] = mhtech[c].str.replace('not sure', 'Not sure')
    mhtech[c] = mhtech[c].str.replace('Some of them', 'Yes')

# make no_emplyees category ordered, and integer while we're at it
def min_emps_map(x):
    if x == 'More than 1000':
        return(1000)
    else:
        pos = x.find('-')
        return(int(x[:pos]))
mhtech['no_employees'] = mhtech['no_employees'].apply(min_emps_map)

# should be no more missing data
# print('missing values')
# print(mhtech.isnull().sum())
# print()

# impute a value for self-employed depending on number of employees
mhtech.loc[mhtech['no_employees'] == 1 & mhtech['self_employed'].isnull(), 'self_employed'] = 'Yes'
mhtech.loc[mhtech['no_employees'] > 1 & mhtech['self_employed'].isnull(), 'self_employed'] = 'No'

# gender has been over-loaded with free text
# print('gender options')
# print(mhtech.gender.unique())
# print()

# cleaning gender data for analysis
# no disrespect, but we have to reduce the number of categories to make a meaningful analysis
# a bigger dataset with greater representation would be a different story
def categoriseGender(Gender):
    gender=str(Gender).lower().replace('(cis)', '').replace('cis', '').strip()
    if re.search('not sure|-|/|\?|trans|ish', gender):
        return('others')
    elif re.search('fema', gender) or gender == 'f':
        return('female')
    elif re.search('ma', gender) or gender == 'm' or gender == 'msle':
        return('male')
    else:
        return('others')

mhtech['gender'] = mhtech['gender'].apply(categoriseGender)

def likertMap(x):
    if x == 'Very difficult' or x == 'Often':
        return(0)
    elif x == 'Somewhat difficult' or x == 'Sometimes':
        return(1)
    elif x == 'Not sure':
        return(2)
    elif x == 'Somewhat easy' or x == 'Rarely':
        return(3)
    elif x == 'Very easy' or x == 'Never':
        return(4)

def yesnoMap(x):
    if x == 'Yes':
        return(1)
    elif x == 'No':
        return(0)
    elif x == 'Not sure':
        return(-1)

def regionMap(x):
    if x in ['United States', 'Canada']:
        return('NA')
    elif x in ['United Kingdom', 'France', 'Netherlands', 'Switzerland',
                'Germany', 'Austria', 'Ireland', 'Belgium']:
        return('CE')
    elif x in ['Sweden', 'Finland', 'Norway', 'Denmark']:
        return('NE')
    elif x in ['Italy', 'Spain', 'Portugal', 'Slovenia', 'Greece', 'Bosnia and Herzegovina', 'Croatia']:
        return('SE')
    elif x in ['Bulgaria', 'Poland', 'Russia', 'Latvia', 'Romania', 'Hungary', 'Moldova', 'Georgia', 'Czech Republic']:
        return('EE')
    elif x in ['Mexico', 'Brazil', 'Costa Rica', 'Colombia', 'Uruguay', 'Bahamas, The']:
        return('CSA')
    elif x in ['Nigeria', 'South Africa', 'Zimbabwe', 'Israel']:
        return('MEAF')
    elif x in ['India', 'China', 'Philippines', 'Thailand', 'Japan', 'Singapore', 'Australia', 'New Zealand']:
        return('APAC')
    else:
        return(x)

mhtech['region'] = mhtech.country.apply(regionMap)
mhtech.drop('country', 1, inplace=True)
mhtech.leave = mhtech.leave.apply(likertMap)
# this will also fix the missing vals
mhtech.work_interfere = mhtech.work_interfere.fillna('Not sure').apply(likertMap)

# code all the binary indep vars
for c in mhtech.columns:
    if c in ['age', 'gender', 'country', 'leave', 'work_interfere', 'comments', 'no_employees', 'region']:
        continue
    else:
        mhtech[c] = mhtech[c].apply(yesnoMap)
        
# should be no more missing data
# print('missing values')
# print(mhtech.isnull().sum())
# print()

mhtech.to_csv('CHIRPS\\datafiles\\mhtech14.csv.gz', index=False, compression='gzip')

In [450]:
file = '2017.csv'
archive = zipfile.ZipFile('CHIRPS/source_datafiles/mhtech.zip', 'r')
lines = archive.read(file).replace(b"\xa0", b" ").decode("utf-8", 'ignore').split('\r\n')
archive.close()
# convert strings to var lists - deal with some free text issues
lines = [lines[i].replace("<strong>", "").replace("</strong>", "").
         replace("Yes - they all did", "all").replace("No - none did", "none").
         replace("Some did", "some").replace("Yes - always", "Always").
         replace("Overall - how", "How").
         replace("I don't know", "Not sure").replace("No - I don't think they would", "No").
         replace("No - I don't think it would", "No").replace("Yes - they do", "Definitely").
         replace("Yes - I think it would", "Yes").replace("Yes - I know several", "Several").
         replace("Yes - it has", "Definitely").replace("No - they do not", "Definitely not").
         replace("Yes - I know several", "Several").replace("I know some", "Some").
         replace("No - I don't know any", "None").replace("Not applicable to me", "N/A").
         replace("No - at none of my previous employers", "No").replace("Some of my previous employers", "Some").
         replace("Sometimes - if it comes up", "Sometimes").replace("No - because it doesn't matter", "No need").
         replace("No - because it would impact me negatively", "No").replace("Yes - I was aware of all of them", "All").
         replace("I was aware of some", "Some").replace("N/A (not currently aware)", "No").
         replace("No - I only became aware later", "No").replace("Yes - I experienced", "Yes").
         replace("Maybe/Not sure", "Not sure").replace("Yes - I observed", "Obs").
         replace("Yes - at all of my previous employers", "Yes").replace("performance - ", "Yes").
         replace("Honestly,I", "I").replace("with ,y", "with my").replace("pay,ents", "payments").
         replace("awareness,more", "awareness - more").replace("crazy,", "crazy").replace("neutral,", "neutral").
         replace("talk,", "talk").replace("1,2,3", "1-2-3").replace("A,B,C", "A-B-C").
         replace(" ,eating", " - eating").replace("grief,coping", "grief - coping").
         split(',') for i in range(len(lines))]
names = [nm.replace('"', '').strip() for nm in lines[0]]
lines = lines[1:]

lines.pop() # empty final row
# for c in range(1):
#     names.append('emty_col' + str(c))
mhtech17 = pd.DataFrame(lines, columns=names)


# mhtech17[[e is not None for e in mhtech17.emty_col0]]
# [c for c in mhtech17[[e is not None for e in mhtech17.emty_col0]].loc[487]]


In [451]:
file = '2018.csv'
archive = zipfile.ZipFile('CHIRPS/source_datafiles/mhtech.zip', 'r')
lines = archive.read(file).replace(b"\xa0", b" ").decode("utf-8", 'ignore').split('\r\n')
archive.close()
# convert strings to var lists - deal with some free text issues
lines = [lines[i].replace("<strong>", "").replace("</strong>", "").
         replace("Yes - they all did", "all").replace("No - none did", "none").
         replace("Some did", "some").replace("Yes - always", "Always").
         replace("Overall - how", "How").
         replace("I don't know", "Not sure").replace("No - I don't think they would", "No").
         replace("No - I don't think it would", "No").replace("Yes - they do", "Definitely").
         replace("Yes - I think it would", "Yes").replace("Yes - I know several", "Several").
         replace("Yes - it has", "Definitely").replace("No - they do not", "Definitely not").
         replace("Yes - I know several", "Several").replace("I know some", "Some").
         replace("No - I don't know any", "None").replace("Not applicable to me", "N/A").
         replace("No - at none of my previous employers", "No").replace("Some of my previous employers", "Some").
         replace("Sometimes - if it comes up", "Sometimes").replace("No - because it doesn't matter", "No need").
         replace("No - because it would impact me negatively", "No").replace("Yes - I was aware of all of them", "All").
         replace("I was aware of some", "Some").replace("N/A (not currently aware)", "No").
         replace("No - I only became aware later", "No").replace("Yes - I experienced", "Yes").
         replace("Maybe/Not sure", "Not sure").replace("Yes - I observed", "Obs").
         replace("Yes - at all of my previous employers", "Yes").replace("performance - ", "Yes").
         replace("Honestly,I", "I").replace("with ,y", "with my").replace("pay,ents", "payments").
         replace("awareness,more", "awareness - more").replace("crazy,", "crazy").replace("neutral,", "neutral").
         replace("talk,", "talk").replace("1,2,3", "1-2-3").replace("A,B,C", "A-B-C").
         replace(" ,eating", " - eating").replace("grief,coping", "grief - coping").
         split(',') for i in range(len(lines))]
names = [nm.replace('"', '').strip() for nm in lines[0]]
lines = lines[1:]

lines.pop() # empty final row
mhtech18 = pd.DataFrame(lines, columns=names)

In [452]:
file = '2016.csv'
archive = zipfile.ZipFile('CHIRPS/source_datafiles/mhtech.zip', 'r')
lines = archive.read(file).decode("utf-8").split('\n')
archive.close()

# convert strings to var lists - deal with some free text issues
lines = [lines[i].replace("<strong>", "").replace("</strong>", "").
         replace("Yes - they all did", "all").replace("No - none did", "none").
         replace("Some did", "some").replace("Yes - always", "Always").
         replace("Overall - how", "How").
         replace("I don't know", "Not sure").replace("No - I don't think they would", "No").
         replace("No - I don't think it would", "No").replace("Yes - they do", "Definitely").
         replace("Yes - I think it would", "Yes").replace("Yes - I know several", "Several").
         replace("Yes - it has", "Definitely").replace("No - they do not", "Definitely not").
         replace("Yes - I know several", "Several").replace("I know some", "Some").
         replace("No - I don't know any", "None").replace("Not applicable to me", "N/A").
         replace("No - at none of my previous employers", "No").replace("Some of my previous employers", "Some").
         replace("Sometimes - if it comes up", "Sometimes").replace("No - because it doesn't matter", "No need").
         replace("No - because it would impact me negatively", "No").replace("Yes - I was aware of all of them", "All").
         replace("I was aware of some", "Some").replace("N/A (not currently aware)", "No").
         replace("No - I only became aware later", "No").replace("Yes - I experienced", "Yes").
         replace("Maybe/Not sure", "Not sure").replace("Yes - I observed", "Obs").
         replace("Yes - at all of my previous employers", "Yes").replace("performance - ", "Yes").
         replace("Honestly,I", "I").replace("with ,y", "with my").replace("pay,ents", "payments").
         replace("awareness,more", "awareness - more").replace("crazy,", "crazy").replace("neutral,", "neutral").
         replace("talk,", "talk").replace("1,2,3", "1-2-3").replace("A,B,C", "A-B-C").
         replace(" ,eating", " - eating").replace("grief,coping", "grief - coping").
         split(',') for i in range(len(lines))]
names = [nm.replace('"', '').strip() for nm in lines[0]]
lines = lines[1:]

# np.array(names)[[("disorder" in n) for n in names]]
lines.pop() # empty final row
mhtech16 = pd.DataFrame(lines, columns=names)



In [3]:
# load what we need
import CHIRPS.datasets as ds
import CHIRPS.reproducible as rp
import CHIRPS.datasets_proprietary as dsp

# CHIRPS default set up
merging_bootstraps = 20
pruning_bootstraps = 20
delta = 0.05

forest_walk_async=False
chirps_explanation_async=False

n_instances = 5

# model = 'RandomForest'
model = 'AdaBoost1'
# model = 'AdaBoost2'
# model = 'GBM'

do_Anchors = True
do_dfrgTrs = True

datasets = [dsp.usoc2] # here can opt for just one, e.g. [rp.datasets[0]] (as an iterator)
start_instance = 0 # here can opt to start at a specific instance

project_dir = 'V:\\whiteboxing\\examples' # defaults to a directory "whiteboxing" in the working directory
# project_dir = 'C:\\Users\\Crutt\\Documents\\whiteboxing\\tests'
random_state_splits = 123 # change this if you want to try different splits of the data into test / train
random_state_rf = 123 # change this if you want to try with different forest construction
random_state_exp = 123 # change this if you want to try with different runs of the explainer algorithm (affects bootstrap eval)

verbose = True

tuning = {'grid' : None, 'override' : False}
if model == 'RandomForest':
    tuning.update({'grid' : None}) # defaults to n_trees [200, 400, ..., 1600]
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=random_state_rf,
                                           random_state_splits=random_state_splits,
                                           start_instance=start_instance, verbose=verbose)

    kwargs = {'support_paths' : 0.1, 'alpha_paths' : 0.5, 'disc_path_bins' : 4,
             'score_func' : 1, 'weighting' : 'chisq',
             'merging_bootstraps' : merging_bootstraps,
             'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta}
 
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'chirps_explanation_async' : chirps_explanation_async}
    
    rp.do_benchmarking(benchmark_items, verbose, **control)
    
    if do_Anchors:
        control = {'method' : 'Anchors',
                   'model' : model,
                   'n_instances' : n_instances,
                   'random_state' : random_state_exp}
        rp.do_benchmarking(benchmark_items, verbose, **control)
        
    if do_dfrgTrs:
        control = {'method' : 'defragTrees',
                    'Kmax' : 1, 'restart' : 1, 'maxitr' : 1,
                    'model' : model,
                    'n_instances' : n_instances,
                    'random_state' : random_state_exp}
        rp.do_benchmarking(benchmark_items, verbose, **control)
            
elif model == "AdaBoost1":
    algo = 'SAMME'
    max_depth = [i for i in range(1, 5)]
    tuning.update({'grid' : {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)], 'algorithm': [algo]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=123, random_state_splits=123,
                                           start_instance=start_instance, verbose=verbose)
    
    kwargs = {'paths_lengths_threshold' : 5,
             'support_paths' : 0.01, 'alpha_paths' : 0.0,
             'disc_path_bins' : 4, 'disc_path_eqcounts' : True,
             'score_func' : 1, 'weighting' : 'kldiv',
             'which_trees' : 'majority',
             'merging_bootstraps' : merging_bootstraps,
             'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta}
 
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'chirps_explanation_async' : chirps_explanation_async}
    
    rp.do_benchmarking(benchmark_items, verbose=True, **control)
    
elif model == 'AdaBoost2':
    algo = 'SAMME.R'
    max_depth = [i for i in range(1, 5)]
    tuning.update({'grid' : {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)], 'algorithm': [algo]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=123, random_state_splits=123,
                                           start_instance=start_instance, verbose=verbose)
    
    kwargs = {'paths_lengths_threshold' : 5,
                 'support_paths' : 0.01, 'alpha_paths' : 0.0,
                 'disc_path_bins' : 8, 'disc_path_eqcounts' : True,
                 'score_func' : 1, 'weighting' : 'kldiv',
                 'which_trees' : 'majority',
                 'merging_bootstraps' : merging_bootstraps,
                 'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta}
    
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'chirps_explanation_async' : chirps_explanation_async}
    
    rp.do_benchmarking(benchmark_items, verbose=True, **control)
    
else: # GBM
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=123, random_state_splits=123,
                                           start_instance=start_instance, verbose=verbose)

Preprocessing usoc2 data and model for usoc2 with random state = 123
Split data into main train-test and build forest


C:\ProgramData\Anaconda3\envs\B3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Train main model
Over-riding previous tuning parameters. New grid tuning... (please wait)

Finding best params with 10-fold CV
0.640 (+/-0.018) for {'algorithm': 'SAMME', 'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 200}
0.640 (+/-0.016) for {'algorithm': 'SAMME', 'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 400}
0.640 (+/-0

Best OOB Accuracy Estimate during tuning: 0.6401
Best parameters:{'algorithm': 'SAMME', 'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 600, 'random_state': 123}


Beginning benchmark for usoc2 data.
Hyper-parameter settings: {'paths_lengths_threshold': 5, 'support_paths': 0.01, 'alpha_paths': 0.0, 'disc_path_bins': 4, 'disc_path_eqcounts': True, 'score_func': 1, 'weighting': 'kldiv', 'which_trees': 'majority', 'merging_bootstraps': 20, 'pruning_bootstraps': 20, 'delta': 0.05}

Prepare Unseen Data and Predictions for CHIRPS benchmark
Walking forest for 5 instances... (please wait)
Forest Walk with async = False
Forest Walk time elapsed: 0.3383 seconds

Runni

C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1337: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(uppers, upper_bins)[0]).round(5) # can result in nans if no value falls into bin
C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1342: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(lowers, lower_bins)[0]).round(5) # can result in nans
C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1337: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(uppers, upper_bins)[0]).round(5) # can result in nans if no value falls into bin
C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1342: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(lowers, lower_bins)[0]).round(5) # can result in nans
C:\Users\id126493\Documents\GitHub\explain_te\CHIRPS\structures.py:1337: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(upper

CHIRPS time elapsed: 25.9282 seconds
CHIRPS with async = False

Evaluating found explanations
Results saved to V:\whiteboxing\tests\usoc2\AdaBoost1_CHIRPS_rnst_123
CHIRPS batch results eval time elapsed: 10.3810 seconds



In [ ]:
# for notebook plotting
%matplotlib inline 

# load what we need
import time
import timeit
import numpy as np
import CHIRPS.structures as strcts
import CHIRPS.routines as rt
import CHIRPS.reproducible as rp
import CHIRPS.boosting_scratch as bs
from CHIRPS import p_count_corrected, if_nexists_make_dir, chisq_indep_test, entropy_corrected, contingency_test, confidence_weight
from IPython.display import Image
from sklearn.externals.six import StringIO
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from pydotplus import graph_from_dot_data

# demo datasets that ship with package. all from UCI unless stated otherwise
# import CHIRPS.datasets as ds
# ds.adult_data, ds.adult_samp_data, ds.adult_small_samp_data Large dataset ships with manageable sub samples
# ds.bankmark_data, ds.bankmark_samp_data
# ds.car_data
# ds.cardio_data this is the cardiotocography dataset
# ds.credit_data
# ds.german_data
# ds.lending_data, ds.lending_samp_data, ds.lending_small_samp_data, ds.lending_tiny_samp_data from Kaggle. see datasets_from_source file for links
# ds.nursery_data, ds.nursery_samp_data
# ds.rcdv_data, ds.rcdv_samp_data from US government see datasets_from_source file for links

### common config - can be ommitted if defaults are OK

In [ ]:
project_dir = 'V:\\whiteboxing\\examples' # defaults to a directory "whiteboxing" in the working directory
# project_dir = 'C:\\Users\\Crutt\\Documents\\whiteboxing\\examples'
random_state_splits = 123 # one off for splitting the data into test / train
random_state = 123 # for everything else - e.g. building a new rf with same data

# Build a Random Forest Model to Predict and Explain
First, a wrapper is created for the dataset. Use one that ships with the package, or create your own.
Then split the data into training and (hold out) test set using the convenience functions in the package. These return an object that contain the split data in various representations, such as Pandas DataFrames and encoded, sparse matrices.

In [ ]:
# load one of the included datasets
# project_dir will default to directory name CHIRPS in the working directory if not given
# random_state will default to 123
mydata = rp.datasets[0](random_state=random_state, project_dir=project_dir)
meta_data = mydata.get_meta()
save_path = meta_data['get_save_path']()

# split the data. here using a basic sampling method.
# the returned object is a wrapper class that contains
# the train and test splits for X and y

# also the the encoded versions of X_train and X_test that the rf will use
# this is because we prefer onehot encoded over allowing categorical vars to be represented as integer
# scikit would treat these as ordinal, which is inappropriate

# also some meta-data: priors for y, the indexes from the input data

# also some convenience functions for leave-one-out testing

# train test split - one off hard-coded random state.
# random state can be ommitted 
# and will default to the state held in the dataset container
# which defaults to 123 if ommitted in the constructor
train_index, test_index = mydata.get_tt_split_idx(random_state=random_state_splits)
# optionally, indexes can be ommitted and will default to scikit's train_test_split method
tt = mydata.tt_split(train_index, test_index)

In [ ]:
# build model
model='GBM'
# tuning_grid = {'max_depth' : [d for d in [i for i in range(1, 5)]],
#               'n_estimators' : [(i + 1) * 200 for i in range(2)],
#               'learning_rate' : [i/10 for i in range(1, 11)],
#               'subsample' : [0.25, 0.5, 0.75, 1.0]}

tuning_grid = {'max_depth' : [d for d in [i for i in range(1, 2)]],
                'n_estimators' : [(i + 1) * 200 for i in range(1, 2)],
                'learning_rate' : [i/10 for i in range(1, 2)],
                'subsample' : [1.0]}


rf = rp.forest_prep(ds_container=tt,
                    meta_data=meta_data,
                    override_tuning=True,
                    model=model,
                    tuning_grid=tuning_grid,
                    save_path=save_path,
                    plot_cm=True, plot_cm_norm=True)


In [ ]:
# control for async processes - each tree walk can be done in its own core
# and so can each explanation (e.g. rule conditions merge by hill-climbing)
# these will default to false if not passed explicitly to the explainer function
# on a multi-core machine there should be a good speed up for large batches
# when the batch_size advantage exceeds the overhead of setting up multi-processing
# timings will be printed to screen so you can see if it helps
forest_walk_async=False
chirps_explanation_async=False

# how many instances to explain in total from a test/unseen set
n_instances = 1

# this will normalise the above parameters to the size of the dataset
n_instances = rt.n_instance_ceiling(ds_container=tt, n_instances=n_instances)

# this gets the next batch out of the data_split_container according to the required number of instances
# all formats can be extracted, depending on the requirement
# unencoded, encoded (sparse matrix is the type returned by scikit), ordinary dense matrix also available
instances, _, instances_enc, instances_enc_matrix, labels = tt.get_next(n_instances, which_split='test') # default

In [ ]:
# get all the model predictions for the test instance(s) we're looking at
preds_idx = labels.index
preds = rf.predict(X=instances_enc)

f_walker = strcts.forest_walker(forest = rf, meta_data=meta_data)

print('Walking forest for ' + str(len(labels)) + ' instances... (please wait)')

# set the timer
forest_walk_start_time = timeit.default_timer()

# do the walk - returns a batch_paths_container (even for just one instance)
# requires the X instances in a matrix (dense, ordinary numpy matrix) - this is available in the data_split_container
bp_container = f_walker.forest_walk(instances = instances_enc_matrix
                        , labels = preds # we're explaining the prediction, not the true label!
                        , forest_walk_async = forest_walk_async)

# stop the timer
forest_walk_end_time = timeit.default_timer()
forest_walk_elapsed_time = forest_walk_end_time - forest_walk_start_time

print('Forest Walk with async = ' + str(forest_walk_async))
print('Forest Walk time elapsed:', "{:0.4f}".format(forest_walk_elapsed_time), 'seconds')

In [ ]:
# get what the model predicts on the training sample
sample_labels = rf.predict(tt.X_train_enc)

# build CHIRPS and a rule for each instance represented in the batch paths container
CHIRPS = strcts.batch_CHIRPS_explainer(bp_container,
                                forest=rf,
                                sample_instances=tt.X_train_enc, # any representative sample can be used
                                # sample_labels=tt.y_train,  # any representative sample can be used
                                sample_labels=sample_labels,
                                meta_data=meta_data)

print('Running CHIRPS on a batch of ' + str(len(labels)) + ' instances... (please wait)')
# start a timer
ce_start_time = timeit.default_timer()

CHIRPS.batch_run_CHIRPS(chirps_explanation_async=chirps_explanation_async,
                        paths_lengths_threshold=5,
                        alpha_paths=0.0,
                        support_paths=0.01,
                        score_func=1,
                        disc_path_bins=4,
                        target_classes=preds,
                        merging_bootstraps=20,
                        pruning_bootstraps=20,
                        delta=0.1,
                        weighting='chisq')

ce_end_time = timeit.default_timer()
ce_elapsed_time = ce_end_time - ce_start_time
print('CHIRPS time elapsed:', "{:0.4f}".format(ce_elapsed_time), 'seconds')
print('CHIRPS with async = ' + str(chirps_explanation_async))

In [ ]:
# iterate over all the test instances to determine the various scores using leave-one-out testing
print('evaluating found explanations')
print()
results_start_time = timeit.default_timer()

rt.evaluate_CHIRPS_explainers(CHIRPS, tt, labels.index, # for batch runs: tt.y_test.index,
                              forest=rf,
                              meta_data=meta_data,
                              model=model,
                              eval_start_time=results_start_time,
                              print_to_screen=True, # set True when running single instances
                              eval_alt_labelings=True,
                              eval_rule_complements=True,
                              save_results_path=save_path,
                              dataset_name='test',
                              save_results_file='CHIRPS' + '_rnst_' + str(random_state),
                              save_CHIRPS=False)

results_end_time = timeit.default_timer()
results_elapsed_time = results_end_time - results_start_time
print('CHIRPS batch results eval time elapsed:', "{:0.4f}".format(results_elapsed_time), 'seconds')
# this completes the CHIRPS runs

In [ ]:
import numpy as np
n_kwargs = 96

disc_path_bins = np.tile(np.tile(np.repeat([4, 8], 3), 8), 2)
disc_path_eqcounts = np.tile(np.tile(np.repeat([True, False], 6), 4), 2)
support_paths = np.tile(np.tile([0.05, 0.02, 0.01], 16), 2)
weighting = np.tile(np.repeat(['chisq', 'kldiv', 'lodds', 'nothing'], 12), 2)
which_trees = np.repeat(['majority', 'conf_weighted'], 48)

kwargs_grid = {k : {'paths_lengths_threshold' : 5, 'alpha_paths' : 0.0,
                    'disc_path_bins' : dpb, 'disc_path_eqcounts' : dpeq,
                    'score_func' : 1, 'weighting' : w, 'support_paths' : sp,
                    'merging_bootstraps' : merging_bootstraps,
                    'pruning_bootstraps' : pruning_bootstraps,
                    'which_trees' : wchtr,
                    'delta' : delta} 
for k, dpb, dpeq, w, sp, wchtr \
               in zip(range(n_kwargs), disc_path_bins, disc_path_eqcounts, weighting, support_paths, which_trees)}

kwargs_grid

In [ ]:
fail

In [ ]:
X = tt.X_test_enc[0:3,]
pred_raw = [confidence_weight(estimator.predict_proba(X), 'proba') for estimator in rf.estimators_]
print(np.mean(pred_raw, axis = 0))
pred_log = [confidence_weight(estimator.predict_proba(X), 'log_proba') for estimator in rf.estimators_]
print(np.mean(pred_log, axis = 0))
pred = [confidence_weight(estimator.predict_proba(X), 'conf_weight') for estimator in rf.estimators_]
print(np.mean(pred, axis = 0))

In [ ]:
pred = [confidence_weight(estimator.predict_proba(X), 'conf_weight') for estimator in rf.estimators_]
pred = sum(pred)
pred /= rf.estimator_weights_.sum()
print(pred)
pred = pred / (len(pred[0]) - 1)
pred = np.exp(pred)
print(pred)
normalizer = pred.sum(axis=1)[:, np.newaxis]
normalizer[normalizer == 0.0] = 1.0
print(normalizer)
pred /= normalizer
pred


In [ ]:
pred = [confidence_weight(estimator.predict_proba(X), 'log_proba') for estimator in rf.estimators_]
pred = np.mean(pred, axis=0)
#pred /= rf.estimator_weights_.sum()
print(pred)
pred = np.exp(pred)
print(pred)
normalizer = pred.sum(axis=1)[:, np.newaxis]
normalizer[normalizer == 0.0] = 1.0
print(normalizer)
pred /= normalizer
pred

In [ ]:
# how is GB working to calculate the predict function
import numpy as np
n = 300
gbpreds = [gb.predict(X_test[i])[0] for i in range(n)]
reg_tots = np.empty(n)
for i in range(n):
    reg_tots[i] = sum([gb.estimators_[j][0].predict(X_test[i])[0] for j in range(ne)])

mn = np.inf
mx = -np.inf
for pred, tot in zip(gbpreds, reg_tots):
    if pred == 0:
        if mx < tot:
            mx = tot
    if pred == 1:
        if mn > tot:
            mn = tot
            
print(mx, mn)

# Preparing unseen data

Again note:
test set has never been "seen" by random forest during training
test set has been only used to assess model (random forest) accuracy - no additional tuning after this
test set has not be involved in generating the explainer

## optional: memory and computation cost management
#### CHIRPS is time economical but memory intensive to compute for lots of instances at once
option 1: choose a smaller number of instances to explain

In [ ]:
# control for async processes - each tree walk can be done in its own core
# and so can each explanation (e.g. rule conditions merge by hill-climbing)
# these will default to false if not passed explicitly to the explainer function
# on a multi-core machine there should be a good speed up for large batches
# when the batch_size advantage exceeds the overhead of setting up multi-processing
# timings will be printed to screen so you can see if it helps
forest_walk_async=False
chirps_explanation_async=False

# the number of instances can be controlled by
# batch_size - how many instances to explain at one time
batch_size = 1
# how many instances to explain in total from a test/unseen set
n_instances = 1

# this will normalise the above parameters to the size of the dataset
n_instances, n_batches = rt.batch_instance_ceiling(ds_container=tt, n_instances=n_instances, batch_size=batch_size)

# this gets the next batch out of the data_split_container according to the required number of instances
# all formats can be extracted, depending on the requirement
# unencoded, encoded (sparse matrix is the type returned by scikit), ordinary dense matrix also available
instances, _, instances_enc, instances_enc_matrix, labels = tt.get_next(batch_size, which_split='test') # default

option 2: just run the whole test set

In [ ]:
# instances = tt.X_test; instances_enc = tt.X_test_enc; instances_enc_matrix = tt.X_test_enc_matrix; labels = tt.y_test

## Make predictions from the decision forest on the unseen data
Important point, no compromise on model accuracy

In [ ]:
# get all the model predictions for the test instance(s) we're looking at
preds_idx = labels.index
preds = rf.predict(X=instances_enc)

# CHIRPS Step 1:
## Extract Tree Prediction Paths
### Fit a forest_walker object to the dataset and decision forest
This is a wrapper will extracts the paths of all the given instances. For CHIRPS, we want a large sample. The whole training set or other representative sample will do.

It can also report interesting statistics (treating the forest as a set of random tree-structured variables).

In [ ]:
# wrapper object needs the decision forest itself and the dataset meta data (we have a convenience function for this)
f_walker = strcts.forest_walker(forest = rf, meta_data=meta_data)

Now the work of extracting all the paths for each instance is done

In [ ]:
print('Walking forest for ' + str(len(labels)) + ' instances... (please wait)')

# set the timer
forest_walk_start_time = timeit.default_timer()

# do the walk - returns a batch_paths_container (even for just one instance)
# requires the X instances in a matrix (dense, ordinary numpy matrix) - this is available in the data_split_container
bp_container = f_walker.forest_walk(instances = instances_enc_matrix
                        , labels = preds # we're explaining the prediction, not the true label!
                        , forest_walk_async = forest_walk_async)

# stop the timer
forest_walk_end_time = timeit.default_timer()
forest_walk_elapsed_time = forest_walk_end_time - forest_walk_start_time

print('Forest Walk with async = ' + str(forest_walk_async))
print('Forest Walk time elapsed:', "{:0.4f}".format(forest_walk_elapsed_time), 'seconds')

# CHIRPS Steps 2-4: 
## Freqent pattern mining of paths.
## Score and sort mined path segments.
## Merge path segments into one rule.

This is a wrapper object that will execute steps 2-4 on all the instance-paths in the batch_paths_container.

Note that true_divide warnings are OK. It just means that a continuous variable is unbounded in some way i.e. no greater/less than discontinuity is used in the CHIRPS explanation.

Note also, here we are using the training set to create the explainers. We could use a different dataset as long as it is representative of the training set that built the decision forest. Most important that we don't use the dataset that we wish to explain.

In [ ]:
# get what the model predicts on the training sample
sample_labels = rf.predict(tt.X_train_enc)

# build CHIRPS and a rule for each instance represented in the batch paths container
CHIRPS = strcts.batch_CHIRPS_explainer(bp_container,
                                forest=rf,
                                sample_instances=tt.X_train_enc, # any representative sample can be used
                                # sample_labels=tt.y_train,  # any representative sample can be used
                                sample_labels=sample_labels,
                                meta_data=meta_data)

print('Running CHIRPS on a batch of ' + str(len(labels)) + ' instances... (please wait)')
# start a timer
ce_start_time = timeit.default_timer()

CHIRPS.batch_run_CHIRPS(chirps_explanation_async=chirps_explanation_async,
                        alpha_paths=0.9,
                        support_paths=0.1,
                        score_func=5,
                        disc_path_bins=4,
                        target_classes=preds,
                        merging_bootstraps=20,
                        pruning_bootstraps=20,
                        delta=0.1,
                        weighting='chisq')

ce_end_time = timeit.default_timer()
ce_elapsed_time = ce_end_time - ce_start_time
print('CHIRPS time elapsed:', "{:0.4f}".format(ce_elapsed_time), 'seconds')
print('CHIRPS with async = ' + str(chirps_explanation_async))

# Viewing and Evaluating CHIRPS explanations
Evaluation is done using unseen data to see how well the explanations generalise. The data_split_container object (tt) has a  leave-one-out function that is used during the routine to ensure that the datum we are explaining is excluded from the evaluation.

In [ ]:
# iterate over all the test instances to determine the various scores using leave-one-out testing
print('evaluating found explanations')
print()
results_start_time = timeit.default_timer()

rt.evaluate_CHIRPS_explainers(CHIRPS, tt, tt.y_test.index,
                              forest=rf,
                              meta_data=meta_data,
                              eval_start_time=results_start_time,
                              print_to_screen=True, # set True when running single instances
                              eval_alt_labelings=True,
                              eval_rule_complements=True,
                              save_results_path=save_path,
                              dataset_name='test',
                              save_results_file='CHIRPS' + '_rnst_' + str(random_state),
                              save_CHIRPS=False)

results_end_time = timeit.default_timer()
results_elapsed_time = results_end_time - results_start_time
print('CHIRPS batch results eval time elapsed:', "{:0.4f}".format(results_elapsed_time), 'seconds')
# this completes the CHIRPS runs

In [ ]:
from pandas import Series
forest = rf
train_pred_labels = Series(forest.predict(tt.X_train_enc), index = tt.y_train.index)
CHIRPS.CHIRPS_explainers[0].evaluate_rule(rule='pruned', sample_instances=tt.X_train_enc, sample_labels=train_pred_labels)

In [ ]:
CHIRPS.CHIRPS_explainers[0].posterior

In [ ]:
preds = rf.predict_proba(X=instances_enc)

In [ ]:
preds

In [ ]:
np.log(preds) - 0.5 * np.log(preds).sum(axis=1)[:, np.newaxis]

In [ ]:
pred = sum(_samme_proba(estimator, n_classes, X)
                       for estimator in self.estimators_)

In [ ]:
pred = sum(_samme_proba(estimator, 2, instances_enc)
                       for estimator in rf.estimators_)

In [ ]:
sum(_samme_proba(rf.estimators_[0], 2, instances_enc))

In [ ]:
rf.estimators_[0].predict(instances_enc)

In [ ]:
dir(rf)

In [ ]:
import math
[math.exp(ew)/(1+math.exp(ew)) - 0.5 for ew in rf.estimator_weights_]

In [ ]:
rf.estimator_errors_

In [ ]:
rf.decision_function(instances_enc)

In [ ]:
instances_enc

In [ ]:
rf.predict(instances_enc)

In [ ]:
rf.predict_proba(instances_enc)

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
le_dict = LabelEncoder().fit(mydata.data['workclass'].unique())

In [ ]:
mydata.data

In [ ]:
categoricals = [i for i, dt in enumerate(mydata.data.dtypes.values) if dt.name == 'object']

In [ ]:
categoricals = [i for i, dt in enumerate(mydata.data.dtypes.values)]
categoricals = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [ ]:
oh_dict = OneHotEncoder(categories='auto').fit(mydata.data)

In [ ]:
oh_dict.categories_

In [ ]:
oh_dict.get_feature_names()

In [ ]:
oh_dict.transform(mydata.data['workclass'])

In [ ]:
le_dict.classes_

In [ ]:
le_dict.transform(mydata.data['workclass'])

In [ ]:
-np.sum(np.log([0.000000000000000000000000000000001, 0.8430962343096225, 0.07322175732217619, 0.08368200836820137]) * \
[0.0, 0.8430962343096225, 0.07322175732217619, 0.08368200836820137])

In [ ]:
np.log([0.00000000000001, 0.9999999782235003, 2.1776499755250584e-08, 0.00000000000000001])

In [ ]:
np.log(0.9999999782235003)

In [ ]:
entropy_corrected([0.95, 0.0, 0.0, 0.05])

In [ ]:
contingency_test([0.4, 0.6], [0.9, 0.1], 'kldiv')

In [ ]:
merging_bootstraps = 20
pruning_bootstraps = 20
delta = 0.1 # prune rule terms if loss of precision no greater than delta

disc_path_bins = np.tile(np.repeat([4, 8], 3), 8)
disc_path_eqcounts = np.tile(np.repeat([True, False], 6), 4)
support_paths = np.tile([0.05, 0.02, 0.01], 16)
weighting = np.repeat(['chisq', 'kldiv', 'lodds', 'nothing'], 12)

kwargs_grid = {k : {'paths_lengths_threshold' : 5, 'alpha_paths' : 0.0,
                    'disc_path_bins' : dpb, 'disc_path_eqcounts' : dpeq,
                    'score_func' : 1, 'weighting' : w, 'support_paths' : sp,
                    'merging_bootstraps' : merging_bootstraps,
                    'pruning_bootstraps' : pruning_bootstraps,
                    'delta' : delta} 
for k, dpb, dpeq, w, sp in zip(range(48), disc_path_bins, disc_path_eqcounts, weighting, support_paths)}
kwargs_grid